<a href="https://colab.research.google.com/github/ArindamRoy23/Makemore/blob/main/makemore_FCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# This notebook will deal with making a fully connected NN

In [17]:
import torch
import torch.nn.functional as F
import matplotlib as plt

%matplotlib inline

In [18]:
words = open('names.txt', 'r').read().splitlines()

In [19]:
# building the character level dictionary
chars = sorted(list(set(''.join(words))))
stoi = {s:i + 1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [20]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words:

  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

In [54]:
def build_dataset(words):
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182778, 3]) torch.Size([182778])
torch.Size([22633, 3]) torch.Size([22633])
torch.Size([22735, 3]) torch.Size([22735])


In [47]:
C = torch.randn((27, 2)).float()
W1 = torch.randn(6,100)
b1 = torch.randn(100)
W2 = torch.randn((100, 27))
b2 = torch.randn(27)
parameters = [C, W1, b1, W2, b2]
lr = 0.1

In [48]:
for p in parameters:
  p.requires_grad = True

In [51]:
for _ in range(10000):
  ix = torch.randint(0, X.shape[0], (32,))
  emb = C[X[ix]]
  h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits, Y[ix])
  # print(loss)
  for p in parameters:
    p.grad = None
  loss.backward()
  for p in parameters:
    p.data = p.data - lr* p.grad

In [52]:
print(loss)

tensor(2.4124, grad_fn=<NllLossBackward0>)


In [29]:
logits.shape
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)
loss = -prob[torch.arange(X.shape[0]), Y].log().mean()
loss

tensor(16.4528)

In [55]:
# With tr only

In [73]:
C = torch.randn((27, 2)).float()
W1 = torch.randn(6,300)
b1 = torch.randn(300)
W2 = torch.randn((300, 27))
b2 = torch.randn(27)
parameters = [C, W1, b1, W2, b2]
lr = 0.01
for p in parameters:
  p.requires_grad = True

In [74]:
for _ in range(30000):
  ix = torch.randint(0, Xtr.shape[0], (300,))
  emb = C[Xtr[ix]]
  h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits, Y[ix])
  # print(loss)
  for p in parameters:
    p.grad = None
  loss.backward()
  for p in parameters:
    p.data = p.data - lr* p.grad

In [75]:
print(loss)

tensor(2.3841, grad_fn=<NllLossBackward0>)


In [77]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
print(loss)

tensor(2.6114, grad_fn=<NllLossBackward0>)
